<a href="https://colab.research.google.com/github/jlavyan/AFNetworking/blob/master/smile_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Smile Test

In [ ]:


# install required modules

!pip install -U -q PyDrive

In [ ]:
# import required modules

import argparse
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# Define the model class

class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)  # Now dynamic, based on input_size
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def predict(self, x):
        # This method can be used during inference to obtain probabilities
        x = self.forward(x)
        return torch.softmax(x, dim=1)

In [ ]:

# Mount Google Drive to access the input file and model file
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:

# Define the input file and model file names
input_file = 'input_file.csv'
model_file = 'model.pth'

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Load the input file and model file from Google Drive
fileCsv = drive.CreateFile({'id':'1b50V1NzkAejK0WhMSqIHZZOwrLSsvjur'})
fileCsv.GetContentFile(input_file)

fileModel = drive.CreateFile({'id':'13-DUV7X6iZNXBOZekg9zoigLZ9rHivn4'})
fileModel.GetContentFile(model_file)

In [ ]:

# Load the input file into a Pandas DataFrame
df = pd.read_csv(input_file)

# Automatically determine feature columns by excluding 'Smiling'
features_columns = df.columns[df.columns != 'Smiling']
num_features = len(features_columns)  # Determine the number of feature columns dynamically

# Load the model with the correct input feature size
read_model = BinaryClassifier(input_size=num_features)
read_model.load_state_dict(torch.load(model_file))
read_model.eval()

# Separate features and labels
features = df[features_columns]
labels = df['Smiling'].replace(-1, 0)  # Replace -1 with 0 in the label column

# Convert features and labels to PyTorch Tensors
inputs = torch.tensor(features.values).float()
actual_labels = torch.tensor(labels.values).long()  # Assuming labels are integer classes

# Evaluate the model
with torch.no_grad():  # No need to track gradients during evaluation
    outputs = read_model(inputs)
    _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability

# Compare predictions to actual labels
correct_predictions = (predicted == actual_labels).sum().item()
total_predictions = predicted.size(0)
accuracy = correct_predictions / total_predictions * 100

# Specific accuracy for positive detections
positive_label = 1  # Assuming '1' is the label for 'Smiling'
true_positive = ((predicted == positive_label) & (actual_labels == positive_label)).sum().item()
possible_positives = (actual_labels == positive_label).sum().item()

if possible_positives > 0:
    accuracy_positives = true_positive / possible_positives * 100
else:
    accuracy_positives = 0.0

# Print or process predictions and accuracy
print("Predictions:", predicted)
print("Actual Labels:", actual_labels)
print(f"Overall Accuracy: {accuracy:.2f}%")
print(f"Accuracy for Detected Positives: {accuracy_positives:.2f}%")

Predictions: tensor([1, 0, 1,  ..., 0, 0, 1])
Actual Labels: tensor([1, 0, 1,  ..., 0, 0, 1])
Overall Accuracy: 90.15%
Accuracy for Detected Positives: 89.06%
